In [2]:
# Montar/treinar o modelo e verificar a precisão

In [4]:
import tensorflow as tf
import keras
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from category_encoders import OneHotEncoder


In [ ]:
df = pd.read_csv("/")
df.head()

In [7]:
df["perigo"] = [
    0.5,
    0.5,
    1,
    0.2,
    0.1
]

NameError: name 'df' is not defined

In [ ]:
## Separação da variável dependente

X = df.drop(columns="perigo")
y = df["perigo"]

In [ ]:
# Isso aqui divide o treino do teste com uma proporção de 0.5 
## Se esse valor ficar muito grande, a eficiência pode ser reduzida porque o modelo começa a decorar os dados

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

In [5]:
cat_bairros_cols = ["RA XX - ÁGUAS CLARAS","ARNIQUEIRA","RA I - BRASILIA","RA IV - BRAZLÂNDIA","RA XIX - CANDANGOLÂNDIA",
            "RA IX - CEILÂNDIA","RA XI - CRUZEIRO","RA XXV - SCIA/ESTRUTURAL","RA XXXI - FERCAL",
            "RA II - GAMA","RA X - GUARÁ","RA XXVIII - ITAPOÃ","RA XXVII - JARDIM BOTÂNICO",
            "RA XVIII - LAGO NORTE","RA XVI - LAGO SUL","RA VIII - NÚCLEO BANDEIRANTE","RA VII - PARANOÁ",
            "RA XXIV - PARK WAY","RA VI - PLANALTINA","RA XV - RECANTO DAS EMAS","RA XVII - RIACHO FUNDO",
            "RA XXI - RIACHO FUNDO II","RA XII - SAMAMBAIA","RA XIII - SANTA MARIA","RA XIV - SÃO SEBASTIÃO",
            "RA XXIX - S.I.A.","RA V - SOBRADINHO","RA XXVI - SOBRADINHO II","SOL NASCENTE","RA XXII - SUDOESTE",
            "RA III - TAGUATINGA","RA XXIII - VARJÃO","RA XXX - VICENTE PIRES"]

In [6]:
cat_crimes_cols = ["HOMICÍDIO","LATROCÍNIO","LESÃO CORPORAL SEG. DE MORTE","ROUBO A TRANSEUNTE",
                   "ROUBO DE VEÍCULO","ROUBO EM COLETIVO","ROUBO EM COMÉRCIO *","ROUBO EM RESIDÊNCIA",
                   "FURTO EM VEÍCULO","TENTATIVA DE HOMICÍDIO","TENTATIVA DE LATROCÍNIO","ESTUPRO",
                   "ESTUPRO DE VULNERÁVEL","FURTO A TRANSEUNTE","TRÁFICO DE DROGAS",
                   "USO E PORTE DE DROGAS","POSSE/PORTE DE ARMA",
                   "LOCALIZAÇÃO DE VEICULO FURTADO OU ROUBADO"]

In [ ]:
# Transformar as variáveis em números para ser compatível com o tensorflow

cat_encoder = OneHotEncoder(cols=cat_crimes_cols+cat_bairros_cols, drop_invariant=True, handle_missing=0)
X_train = cat_encoder.fit_transform(X_train)
X_test = cat_encoder.transform(X_test)

# Rodar isto opcionalmente caso tiver strings nas variáveis do tf

In [ ]:
X_test.head()

In [ ]:
# Imputação dos dados
## Isso aqui transforma os valores entre 0 e 1 

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
tf.random.set_seed(2)
# Entrada = Bairro, Trimestre (Média), Tipo de crime
#

# Construção das camadas
inp = keras.layers.Input((X_train_scaled.shape[1], ))
hid = keras.layers.Dense(10, activation="relu")(inp)

drop = keras.layers.Dropout(0.5)(hid)

out_soft = keras.layers.Dense(len(y_train), activation="softmax")(drop)
out_sig = keras.layers.Dense(1, activation="sigmoid")(drop)


model_sig = keras.Model(inp, out_sig)
model_soft = keras.Model(inp, out_soft)

model_sig.compile(optimizer='adam',
              loss='msle',
              metrics=['accuracy'])
model_soft.compile(optimizer='adam',
              loss='msle',
              metrics=['accuracy'])

model_sig.summary()
model_soft.summary()

In [ ]:
es = keras.callbacks.EarlyStopping(
    monitor="val_loss", min_delta=0, patience=10, mode="min", restore_best_weights=True,
)

In [ ]:
model_soft.fit(X_train_scaled, y_train, validation_data=(X_test_scaled, y_test), epochs=10, shuffle=True, batch_size=32, callbacks=[es])

In [ ]:
# Testar modelo

loss, accuracy = model_soft.evaluate(X_test_scaled, y_test)
print(f'Acurácia no conjunto de teste: {accuracy}')